In [ ]:
import os
import cv2
import numpy as np
import histomicstk as htk
import scipy as sp
import skimage.io
import skimage.measure
import skimage.color
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image

from glob import glob
from os.path import expanduser, join, basename

plt.rcParams['figure.figsize'] = 15,15
plt.rcParams['image.cmap'] = 'gray'
titlesize = 24

In [ ]:
# Reading in color default. Change to zero for grayscale
def read_image(filename,mode=1):
    try:
        img = cv2.imread(filename,mode)
    except e:
        raise e
    if img is None:
        print('Check your path. Not a file path. Probably a folder path.')
    return img

def write_image(filename,img):
    try:
        cv2.imwrite(filename,img)
    except e:
        raise e
        
def image_check(image_name):
    if image_name.split('.')[-1] not in ['jpg','jpeg','png']:
        return False
    return True
    
def cal_iou(img1, img2):
    intersection = np.sum(img1*img2)
    union = np.sum(img1 + img2) - intersection
    
    return intersection/union

In [ ]:
def segment(image_path, ref_path, output_path, gt_path):
    
    #Constants:
    foreground_threshold = 200
    min_radius = 5
    max_radius = 10
    min_nucleus_area = 200
    
    #reading images
    imInput = skimage.io.imread(image_path)[:, :, :3]
    imReference = skimage.io.imread(ref_path)[:, :, :3]
    gt = skimage.io.imread(gt_path)[:,:,0]

    # Get the mean and standard deviation of reference image in LAB colorspace
    meanRef, stdRef = htk.preprocessing.color_conversion.lab_mean_std(imReference)

    # Perform Reinhard Color Normalization on Input Image
    imNmzd = htk.preprocessing.color_normalization.reinhard(imInput,meanRef,stdRef)

    # Macenko PCA to get the stain vectors 
    I_0 = 255
    w_est = htk.preprocessing.color_deconvolution.rgb_separate_stains_macenko_pca(imNmzd,I_0=255)

    # Perform the deconvolution using the stain vectors
    deconv_result = htk.preprocessing.color_deconvolution.color_deconvolution(imInput, w_est,I_0)

    #stains = ['hematoxylin','eosin','null']
    #In this case we are taking eosin, that is why 1, else 0 in next line
    imNucleiStain = deconv_result.Stains[:,:,1]

    # Now we do segmentation based on a fixed intensity value.
    imFgndMask = sp.ndimage.morphology.binary_fill_holes(imNucleiStain<foreground_threshold)

    imLog = htk.filters.shape.clog(imNucleiStain, imFgndMask,\
                                   sigma_min=min_radius * np.sqrt(2),\
                                   sigma_max=max_radius * np.sqrt(2))

    # detect and segment nuclei using local maximum clustering
    local_max_search_radius = 10
    imNucleiSegMask1, Seeds, Max = htk.segmentation.nuclear.max_clustering(imLog[0], imFgndMask, local_max_search_radius)

    # filter out small objects
    imNucleiSegMask = htk.segmentation.label.area_open(imNucleiSegMask1, min_nucleus_area).astype(np.int)

    imNucleicompact = htk.segmentation.label.compact(imNucleiSegMask, compaction=3)
    k= (imNucleicompact==-1)
    imNucleicompact1=np.copy(k)
    # plt.rcParams['figure.figsize'] = 15,15    
    for ii in range(0,imNucleicompact.shape[0]):
        for jj in range(0,imNucleicompact.shape[1]):
            if imNucleicompact[ii,jj]>0:
                imNucleicompact1[ii,jj]=1

    imNucleicompact2 = skimage.measure.label(imNucleicompact1,connectivity = 1)
    # imInput2 = np.copy(imNmzd)
    # plt.rcParams['figure.figsize'] = 1, 1

    mask_generated = np.copy(imNucleicompact2)
    mask_generated[mask_generated>0] = 1
    gt[gt>0] = 1

    iou_cal = cal_iou(gt, mask_generated)
    cv2.imwrite(output_path+image_path.split('/')[-1],mask_generated)
    
    return iou_cal

In [ ]:
image_path = "Input_Images/01_1.png"
ref_path = "BM_GRAZ_HE_0007_01.png"
output_path = "Output_Images/"
gt_path = "Ground_Truth/01_1.png"

iou = segment(image_path, ref_path, output_path, gt_path)

print(iou)

In [ ]:
iou_list = []
ref_path = "BM_GRAZ_HE_0007_01.png"

input_dir = "Input_Images"
gt_dir = "Ground_Truth"
output_dir = "Output_Images"

inputs = glob(join(input_dir, '*.png'))
gts = glob(join(gt_dir, '*.png'))

if len(inputs) == len(gts):
    for i in range(len(inputs)):
        print("Processing {}".format(inputs[i]))
        iou = segment(inputs[i], ref_path, output_dir, gts[i])
        iou_list.append(iou)
    print("Completed processing all images")    

